# Design Flowsheet for NGCC + SOEC with Steam Integration

This flowsheet example provides a design point model for integration of an SOEC for hydrogen production with an NGCC.  The NGCC nominally produces 650 MW net, and includeds 97% CO2 capture.  The SOEC unit model used here is only approriate for design point calculations.  Simulations of off-desing cases require it to be replaced by a more detailed model.  The SOEC desing point model is based on water utilization and basic mass and energy balances.  The numebr of cells is etimated from a given cell sized and a current density estimate provided by the user. 

This flowsheet can be used to determine design point performance, and to optimize the flowsheet design.  

## Import Required Modules

In [1]:
import os
import pytest
from IPython.core.display import SVG
import pyomo.environ as pyo
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes
import idaes.core.util.scaling as iscale
import idaes.core.util as iutil
import ngcc_soec_design

## Set Global Solver Options

Setting global solver options applies them to any solver created subsequently, including the ones used for initialization.  The used scaling option disables Ipopt's automatic scaling and allows it to uses provided variable scaling.

In [2]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
solver = pyo.SolverFactory("ipopt")

## Create and Initialize the NGCC + SOEC Model

In [3]:
m = pyo.ConcreteModel()
m.fs = ngcc_soec_design.NgccSoecDesignFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)
m.fs.initialize(
    load_from="ngcc_soec_design_init.json.gz",
    save_to="ngcc_soec_design_init.json.gz"
)

## Display the flowsheet section and save them to files

In [ ]:
def display_pfd():
    print("\n\nGas Trubine Section\n")
    display(SVG(m.fs.ngcc.gt.write_pfd()))
    print("\n\nHRSG Section\n")
    display(SVG(m.fs.ngcc.hrsg.write_pfd()))
    print("\n\nSteam Turbine Section\n")
    display(SVG(m.fs.ngcc.st.write_pfd()))
    print("\n\nSOEC Section\n")
    display(SVG(m.fs.soec.write_pfd()))

m.fs.ngcc.gt.write_pfd(fname="data_pfds/gt_soec_design.svg")
m.fs.ngcc.hrsg.write_pfd(fname="data_pfds/hrsg_soec_design.svg")
m.fs.ngcc.st.write_pfd(fname="data_pfds/st_soec_design.svg")
m.fs.soec.write_pfd(fname="data_pfds/soec_soec_design.svg")

display_pfd()

In [ ]:
m.fs.ngcc.lhv_efficiency.display()
m.fs.ngcc.net_power.display()

## Check a few values

In [ ]:
assert pyo.value(m.fs.ngcc.lhv_efficiency[0]) == pytest.approx(0.52, rel=0.01)
assert pyo.value(m.fs.soec.hstrm06.destination.flow_mol[0]) == pytest.approx(853.0, rel=0.01)

## Example: optimize some parameters

Since the amount of steam and water utiliation are fixed and the gas turbine power is fixed, we can maximize the efficeny by maximizing the power out of the system.  The fuel use and hydrogen generation will be fixed.  The power output is negative, so minizing the net power is equivalent to maximizing efficiency.

In [ ]:
m.fs.obj = pyo.Objective(expr=(m.fs.ngcc.net_power[0] + m.fs.soec.total_electric_power[0])/1e6)

In [ ]:
decision_vars = []
def make_decision_var(v, lb, ub):
    v.unfix()
    v.setlb(lb)
    v.setub(ub)
    decision_vars.append(v)

make_decision_var(m.fs.soec.sweep_recycle_split.split_fraction[0, "out"], 0.80, 0.97)
make_decision_var(m.fs.soec.feed_recycle_split.split_fraction[0, "out"], 0.80, 0.97)
make_decision_var(m.fs.soec.water_split.split_fraction[0, "outlet1"], 0.4, 0.6)
make_decision_var(m.fs.soec.sweep_turbine.control_volume.properties_out[0].pressure, 1.2e5, 2.5e5)
m.fs.soec.sweep_turbine.ratioP[0].unfix()
make_decision_var(m.fs.soec.water_heater02.area, 1400, 3000)
make_decision_var(m.fs.soec.water_heater01.area, 1200, 2000)
solver.solve(m, tee=True)

In [ ]:
print(f"Overall net power: {-pyo.value(m.fs.obj)} MW")
for v in decision_vars:
    print(f"{v} {pyo.value(v)}")

## Check value

Since we are not accounting for capital cost here, the heat exchanger area should be at the upper bound.

In [ ]:
assert pyo.value(m.fs.soec.water_heater02.area) == pytest.approx(3000, rel=0.01)

## See PFDs after optimization

In [ ]:
display_pfd()